In [54]:
import pandas as pd
import numpy as np
import datetime

## Считывание файла

In [55]:
transactions = pd.read_csv("e:/MachineLearning/resto-asap2018/casc-resto.csv", sep= ";" , decimal= ",")


In [56]:
clients = pd.read_csv("e:/MachineLearning/resto-asap2018/CASC_Constant.csv" , sep=",")

In [57]:
transactions["RKDate"]=pd.to_datetime(transactions['RKDate']) #приведение типа
transactions.head()

,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage
0,2898197,391,2017-07-16,Москва и Московская область,TGI FRIDAYS,NON ALCOHOL,2,2.0,2.0
1,2903215,43,2015-04-07,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.0
2,2748887,43,2015-05-22,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.0
3,2862077,46,2015-03-05,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.0
4,2862077,46,2015-03-11,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.0,5.0


In [58]:
startdate=datetime.datetime(2017,7,1) #2017-07-01 датa на которую считаются характеристики клиентов


In [87]:
 groupcustomer=transactions.groupby(['CustomerID'])
# Переменная Y, которую требуется предсказывать
def y (result):
        return len(result.loc[(startdate <= result['RKDate']) & (datetime.datetime(2017,12,31) >= result['RKDate'])]) > 0
    # Количество дней до последнего визита клиента перед зафиксированной датой 2017-07-01
def Recency (result):
        return pd.Timedelta(startdate - result[startdate > result['RKDate']]['RKDate'].max()).days
     # Средняя частота походов клиентов на дату 2017-07-01
def Frequency (result):
         return result[result['RKDate'] < startdate].groupby([result[result['RKDate'] < startdate]['RKDate'].dt.year, result[result['RKDate'] < startdate]['RKDate'].dt.month]).size().mean()
        # Средний чек клиента по всем покупкам до 2017-07-01
def Monetaryvalue (result):
        return result[startdate > result['RKDate']]['SummAfterPointsUsage'].mean()
    #Результирующая таблица
def get_result(result):
        return pd.Series({ 
        'Y':  y(result),
        'Recency':Recency(result),
        'Frequency': Frequency(result),
        'Monetary_value':Monetaryvalue(result)
    })


statistics = groupcustomer.apply(get_result)
statistics.head()

,Y,Recency,Frequency,Monetary_value
CustomerID,,,,
2728046,False,160.0,8.142857,251.789474
2728088,False,720.0,17.000000,279.647059
2728089,True,15.0,6.600000,398.734848
2728095,False,177.0,9.000000,262.737374
2728107,True,115.0,5.250000,393.714286


In [25]:
clients.head()

,CustomerId,ActivationDate,Age,Sex,SubscribedEmail,SubscribedPush
0,2728183,2015-01-01,21.0,Female,False,True
1,2728198,2015-01-01,21.0,Female,True,True
2,2728306,2015-01-01,21.0,Female,True,True
3,2728178,2015-01-01,22.0,Male,True,True
4,2728322,2015-01-01,22.0,Male,True,True


# объединение таблиц

In [53]:
resulttable = pd.merge(right=clients[['CustomerId','Sex', 'Age']], left=statistics , left_on='CustomerID', right_on='CustomerId', how='right')
resulttable.head()

,Flag,Recency,Frequency,Monetary_value,Times_using_discount,CustomerId,Sex,Age
0,False,160.0,8.142857,251.789474,3,2728046,Female,24.0
1,False,720.0,17.000000,279.647059,0,2728088,Male,46.0
2,True,15.0,6.600000,398.734848,16,2728089,Female,27.0
3,False,177.0,9.000000,262.737374,42,2728095,Female,54.0
4,True,115.0,5.250000,393.714286,7,2728107,Male,48.0


In [49]:
from sklearn.model_selection import train_test_split 
train, test = train_test_split(resulttable, test_size=0.2) #разбиение клиентов
test.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1760 entries, 9359 to 5641
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Flag                  1760 non-null   bool   
 1   Recency               1760 non-null   float64
 2   Frequency             1760 non-null   float64
 3   Monetary_value        1760 non-null   float64
 4   Times_using_discount  1760 non-null   int64  
 5   CustomerId            1760 non-null   int64  
 6   Sex                   1760 non-null   object 
 7   Age                   1760 non-null   float64
dtypes: bool(1), float64(4), int64(2), object(1)
memory usage: 111.7+ KB


In [ ]:
## https://github.com/romatarasov/lab1Python